In [52]:
# CS180 (CS280A): Project 1 starter Python code

# these are just some suggested libraries
# instead of scikit-image you could use matplotlib and opencv to read, write, and display images

import numpy as np
import skimage as sk
import skimage.io as skio
import logging
import matplotlib.pyplot as plt
import skimage.exposure as exposure
import skimage.color as color
import os

# set up logging
# levels are DEBUG, INFO, WARNING, ERROR, CRITICAL
logging.basicConfig(level=logging.DEBUG)


In [53]:
def load_image_as_float(imname):
    # concatenate the image name with the path
    imname = '../../data/' + imname
    im = skio.imread(imname)
    # convert to float (might want to do this later on to save memory)
    im = sk.img_as_float(im)
    return im

In [54]:
# SPLITTING THE IMAGE

# divide the image into 3 parts (BGR)
def split_image(im):
    # get the height of each part
    # number of rows in the image
    rows = im.shape[0]
    print("rows: ", rows)
    height = int(rows // 3.0)
    print("height: ", height)
    # split the image into 3 parts
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]
    return b, g, r

In [55]:
# SIMPLE ALIGN ALGORITHM

# calculate the error between 2 images
def calculate_error_L2(im1, im2):
    # compute the error
    error = np.sqrt(np.sum((im1 - im2)**2))
    return error

def calculate_error_NCC(im1, im2):
    # dot product between (image1./||image1|| and image2./||image2||).
    # compute the error
    error = np.dot(im1, im2) / (np.linalg.norm(im1) * np.linalg.norm(im2))
    return error

def calculate_shift(baseImage, im2, window_size=20, base_shift=(0,0), error_type='L2'):
    # calculate the base shift
    base_row, base_col = base_shift
    # slice border off the image to not consider border discoloration
    border_size = 20
    baseImage = baseImage[border_size:-border_size, border_size:-border_size]
    im2 = im2[border_size:-border_size, border_size:-border_size]
    # exhaustively search for the best shift
    best_shift = None
    best_error = np.inf
    for row_shift in range(base_row - window_size, base_row + window_size + 1):
        for col_shift in range(base_col - window_size, base_col + window_size + 1):
           # shift the image
                # np.roll(array, shift, axis)
            shifted_im2 = np.roll(im2, (row_shift, col_shift), axis=(0, 1))
            # compute the error
            if error_type == 'L2':
                error = calculate_error_L2(baseImage, shifted_im2)
            elif error_type == 'NCC':
                error = calculate_error_NCC(baseImage, shifted_im2)
            # logging.debug(f"Shift: {row_shift}, {col_shift}, Error: {error}")
            # print("shift: ", row_shift, col_shift)
            # update the best shift and error
            if error < best_error:
                best_error = error
                best_shift = (row_shift, col_shift)
    # return the best shift
    # print("best_shift: ", best_shift)
    # print("best_error: ", best_error)
    return best_shift




In [62]:
# ALIGNING THE IMAGES

# functions that might be useful for aligning the images include:
# np.roll, np.sum, sk.transform.rescale (for multiscale)
def align_image_simple(baseImage, im2, shift=None):
    # Use provided shift if available, otherwise calculate it
    if shift is None:
        shift = calculate_shift(baseImage, im2)
    print("final shift: ", shift)
    # shift im2
    shifted_im2 = np.roll(im2, shift, axis=(0, 1))
    return shifted_im2

# align the bottom 2 images with the top image (BGR)
def align_images_simple(b, g, r):
    # align the bottom 2 images with the top image (BGR)
    ag = align_image_simple(b, g)
    ar = align_image_simple(b, r)
    return ag, ar

In [63]:
# CONSTRUCTING THE PYRAMID

# downsample the image by a factor of 2 at each level and appen to pyramid array
# return the pyramid in order from coarsest to finest
def construct_pyramid(im):
    pyramid = [im] # start with the original image
    levels = 5
    for i in range(levels):
        im = sk.transform.rescale(im, 0.5, anti_aliasing=True)
        pyramid.append(im)
    # logging.info(f"size of end pyramid: {pyramid[-1].shape}")
    #display the image
    # skio.imshow(pyramid[0])
    # skio.show()
    return pyramid[::-1]

# CALCULATE SHIFT PYRAMID
# align the images at each level of the pyramid
def calculate_shift_pyramid(refImage, im2):
    # convert to edges
    refImage = convert_to_edges(refImage)
    im2 = convert_to_edges(im2)
    refPyramid = construct_pyramid(refImage)
    pyramid2 = construct_pyramid(im2)
    base_shift = (0,0)

    for i in range(len(refPyramid)):
        # calculate the shift
        shift = calculate_shift(refPyramid[i], pyramid2[i], 5, base_shift=base_shift)
         # scale the shift for the next level, except for the last iteration
        if i < len(refPyramid) - 1:
            base_shift = tuple(s * 2 for s in shift)
        else:
            # For the last iteration (finest level), don't scale the shift
            base_shift = shift
    return base_shift

def convert_to_edges(im):
    # convert to edges
    im = sk.filters.sobel(im)
    # display the image
    # skio.imshow(im)
    # skio.show()
    return im

# ALIGN PYRAMID
def align_image_pyramid(refImage, im2):
    shift = calculate_shift_pyramid(refImage, im2)
    print("final shift: ", shift)
    # shift im2
    shifted_im2 = np.roll(im2, shift, axis=(0, 1))
    return shifted_im2

def align_images_pyramid(b, g, r):
    ag = align_image_pyramid(b, g)
    ar = align_image_pyramid(b, r)
    return ag, ar

In [64]:
# 
def histogram_equalization(image):
    return exposure.equalize_hist(image)

def adaptive_histogram_equalization(image, kernel_size=None):
    if kernel_size is None:
        # Default kernel size is 1/8 of image height or width, whichever is smaller
        kernel_size = max(image.shape) // 8
    
    return exposure.equalize_adapthist(image, kernel_size=kernel_size)

In [65]:
# AUTOMATIC WHITE BALANCE
# estimate the illuminant
def estimate_illuminant(im):
    # estimate the illuminant
    return np.mean(im, axis=(0, 1))

def apply_white_balance(im):
    # estimate the illuminant
    illuminant = estimate_illuminant(im)
    # Compute scaling factors to shift illuminant towards white (1, 1, 1)
    scaling_factors = 1 / illuminant
    
    # Normalize scaling factors to preserve overall brightness
    scaling_factors /= np.max(scaling_factors)
    
    # Apply correction
    balanced_image = im * scaling_factors
    
    # Clip values to ensure they remain in [0, 1] range
    return np.clip(balanced_image, 0, 1), illuminant


In [72]:


# name of the input file
# imname = 'cathedral.jpg'
# imname = 'monastery.jpg'
# imname = 'church.tif'

# iterate through all the images in the data folder

def process_image(imname):
    # print the name of the image
    print("image name: ", imname)

    # read in the image
    im = load_image_as_float(imname)
        
    # split the image into 3 parts
    b, g, r = split_image(im)
    # show the 3 parts with titles
    # skio.imshow_collection([b, g, r])
    # skio.show()

    

    # align the images
    # use simple alignment if jpg, otherwise use pyramid alignment
    if imname.endswith('.jpg'):
        ag, ar = align_images_simple(b, g, r)
    else:
        ag, ar = align_images_pyramid(b, g, r)
    # create a color image by stacking the aligned images
    # create a 0 matrix with the same shape as the image
    # zeroTest = np.zeros_like(ag)

    im_out = np.dstack([ar, ag, b])

    # # display the image
    # skio.imshow(im_out)
    # skio.show()

    # # edge detection
    # skio.imshow(convert_to_edges(im_out))
    # skio.show()

    # im_pre_hist = im_out.copy()
    # # convert to jpg
    # im_pre_hist = sk.img_as_ubyte(im_pre_hist)
    # # save the image
    # fname = '../../outputs/' + imname.split('.')[0] + '_pre_hist.jpg'
    # skio.imsave(fname, im_pre_hist)

    
    
    # skio.imshow(im_out)
    # skio.show()
    # # histogram equaliztion
    # im_copy = im_out.copy()
    # im_copy = histogram_equalization(im_copy)
    # skio.imshow(im_copy)
    # skio.show()

    # adaptive histogram equalization
    im_out = adaptive_histogram_equalization(im_out)
    # skio.imshow(im_out)
    # skio.show()

    # automatic white balance
    im_out, illuminant = apply_white_balance(im_out)
    # skio.imshow(im_out)
    # skio.show()

    # illuminant_display = np.full((100, 100, 3), illuminant)
    # skio.imshow(illuminant_display)
    # skio.show()

    # # convert to jpg
    # im_out = sk.img_as_ubyte(im_out)
    # # save the image
    # fname = '../../outputs/' + imname.split('.')[0] + '_out.jpg'
    # skio.imsave(fname, im_out)


# process_image('church.tif')
# process_image('plants.tif')
# process_image('garden.tif')
# process_image('flower_bush.tif')
for imname in os.listdir('../../data'):
    process_image(imname)

image name:  emir.tif
rows:  9627
height:  3209
final shift:  (49, 23)
final shift:  (107, 40)
image name:  monastery.jpg
rows:  1024
height:  341
final shift:  (-3, 2)
final shift:  (3, 2)
image name:  church.tif
rows:  9607
height:  3202
final shift:  (25, -2)
final shift:  (61, -14)
image name:  three_generations.tif
rows:  9629
height:  3209
final shift:  (54, 0)
final shift:  (111, 7)
image name:  flower_bush.tif
rows:  9752
height:  3250
final shift:  (48, -8)
final shift:  (96, -24)
image name:  melons.tif
rows:  9724
height:  3241
final shift:  (78, 5)
final shift:  (177, 11)
image name:  onion_church.tif
rows:  9646
height:  3215
final shift:  (52, 24)
final shift:  (107, 35)
image name:  train.tif
rows:  9715
height:  3238
final shift:  (41, 1)
final shift:  (85, 29)
image name:  tobolsk.jpg
rows:  1024
height:  341
final shift:  (3, 3)
final shift:  (6, 3)
image name:  icon.tif
rows:  9732
height:  3244
final shift:  (40, 16)
final shift:  (90, 23)
image name:  cathedral.jpg